In [64]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
from pathlib import Path
import os
import glob
import numpy

In [2]:
folder = 'D:/Darkweb Data/dnmarchives/silkroad2.tar/silkroad2/2014-02-24/categories/*'


In [3]:
# Rename files in directory so that they have the correct file extension.
def converthtmlsr2(folder):
    """Changes file type to from xxxx to xxxx.html so that the files can be read in correctly"""
    for filename in glob.iglob(os.path.join(folder, 'ite*')):
      os.rename(filename, filename + '.html')

#converthtmlsr2(folder)

In [ ]:
# Coinbase API

In [4]:
def cleanhtmlsr2(filename):
  """Extracts relevant information from SilkRoad2 html files.
  Resulting dataframe contains:
  Title, Seller ID, priceUSD, priceBTC, Rating, Reviews, Origin, Destination, Category, Subcategory, Market Name and Date Scraped"""
  with open(filename, encoding='utf8') as infile:  # Open file to support utf8 encoding
    raw_html = BeautifulSoup(infile, "lxml")
  title_raw = raw_html.find_all('div', {'class' :'item_title'})
  cleantags = re.compile('<.*?>')  # Regex for html tags
  title = re.sub(cleantags, '', str(title_raw)).strip().split(', ')  # Strip html tags
  details_raw = raw_html.find_all('div', {'class' :'item_details'})
  details = re.sub(cleantags, '', str(details_raw)).strip().split('\n    ')
  sellerid_full = details[1::4]
  sellerid = [i.split(': ', 1)[1] for i in sellerid_full]
  origin_full = details[2::4]
  origin = [i.split(': ', 1)[1] for i in origin_full]
  destination_full = details[3::4]
  destination = [i.split(': ', 1)[1] for i in destination_full]
  price_raw = raw_html.find_all('div', {'class' :'price_big'})
  priceBTC = str(price_raw).replace('[','').replace(']','').replace('<div class="price_big">฿','').replace('</div>','').split(', ')
  rating = []  # Seller Rating. Out of 5.
  reviews = [] # The number of customer reviews
  priceUSD = [] # Price in American Dollars
  category_raw = os.path.abspath(os.path.join(filename, os.pardir))
  category_raw =  re.sub(r'^.*\\', '',category_raw).replace('drugs-','').capitalize() + '-None'
  category = category_raw.split('-')[0]
  category = [category] * len(title) # Fill column with single category value
  subcat = category_raw.split('-')[1].capitalize()
  subcat = [subcat] * len(title)
  market = ['SilkRoad2'] * len(title) # Fill column with single market name value
  date = [str(Path(filename).parents[2])[-10:]] * len(title)  # # Fill column with single date value
  df = pd.DataFrame([title,sellerid,priceUSD,priceBTC,rating,reviews,origin,destination,category,subcat,market,date]).transpose()  # Create df and transpose
  df.columns = ['Title','Sellerid','PriceUSD','PriceBTC','Rating','Reviews','Origin','Destination','Category','Subcategory','Market','Date']  # Assign column names to df
  return df

#cleanhtmlsr2('D:/Darkweb Data/dnmarchives/silkroad2.tar/silkroad2/2013-12-20/categories/art/items.html')
#cleanhtmlsr2('D:/Darkweb Data/dnmarchives/silkroad2.tar/silkroad2/2013-12-20/categories/drugs-dissociatives-dioscorea/items.html')
#cleanhtmlsr2('D:/Darkweb Data/dnmarchives/silkroad2.tar/silkroad2/2013-12-20/categories/drugs-stimulants/items.html')

In [95]:
#works for final date

def cleanhtmlsr2part2(filename):
  """Extracts relevant information from SilkRoad2 html files.
  Resulting dataframe contains:
  Title, Seller ID, priceUSD, priceBTC, Rating, Reviews, Origin, Destination, Category, Subcategory, Market Name and Date Scraped"""
  with open(filename, encoding='utf8') as infile:  # Open file to support utf8 encoding
    raw_html = BeautifulSoup(infile, "lxml")
  title_raw = raw_html.find_all('div', {'class' :'item_title'})
  cleantags = re.compile('<.*?>')  # Regex for html tags
  title = re.sub(cleantags, '', str(title_raw)).strip().split(', ')  # Strip html tags
  sellerid_raw = raw_html.find_all('div', {'class' :'vendor'})
  sellerid_full = [i.text for i in sellerid_raw]
  sellerid_name = [i.split('by ')[1] for i in sellerid_full]
  sellerid = [i.split('\n')[0] for i in sellerid_name]
  shipping_full = raw_html.find_all('div', {'class' : 'shipping'})
  shipping = [i.text for i in shipping_full]
  origin_full = [i.split('\n ')[1] for i in shipping]
  origin = [i.split(': ', 1)[1] for i in origin_full]
  destination_full = [i.split('\n ')[2] for i in shipping]
  destination = [i.split(': ', 1)[1] for i in destination_full]
  price_raw = raw_html.find_all('div', {'class' : 'price'})
  priceBTC = [i.text for i in price_raw]
  priceBTC = [i.replace('฿','') for i in priceBTC]
  rating_full = raw_html.find_all('span', {'class' : 'vendor_rating average'})  # Seller Rating. Out of 5.
  rating = [i.split('\n')[2] for i in sellerid_full]
  for i in range(0, len(rating)): 
    rating[i] = int(rating[i])/20
  reviews_full = raw_html.find_all('div', {'class' : 'rating_count'}) # The number of customer reviews
  reviews = [i.text for i in reviews_full]
  for i in range(0, len(reviews)): 
    reviews[i] = reviews[i].replace('(','').replace(')','')
    reviews[i] = int(reviews[i])
  priceUSD = [] # Price in American Dollars
  category_raw = os.path.abspath(os.path.join(filename, os.pardir))
  category_raw =  re.sub(r'^.*\\', '',category_raw).replace('drugs-','').capitalize() + '-None'
  category = category_raw.split('-')[0]
  category = [category] * len(title) # Fill column with single category value
  subcat = category_raw.split('-')[1].capitalize()
  subcat = [subcat] * len(title)
  market = ['SilkRoad2'] * len(title) # Fill column with single market name value
  date = [str(Path(filename).parents[2])[-10:]] * len(title)  # # Fill column with single date value
  df = pd.DataFrame([title,sellerid,priceUSD,priceBTC,rating,reviews,origin,destination,category,subcat,market,date]).transpose()  # Create df and transpose
  df.columns = ['Title','Sellerid','PriceUSD','PriceBTC','Rating','Reviews','Origin','Destination','Category','Subcategory','Market','Date']  # Assign column names to df
  return df

cleanhtmlsr2part2('D:/Darkweb Data/dnmarchives/silkroad2.tar/silkroad2/2014-11-05/categories/art/items.html')
cleanhtmlsr2part2('D:/Darkweb Data/dnmarchives/silkroad2.tar/silkroad2/2014-11-05/categories/drugs-dissociatives-dioscorea/items.html')
cleanhtmlsr2part2('D:/Darkweb Data/dnmarchives/silkroad2.tar/silkroad2/2014-11-05/categories/drugs-stimulants/items.html')

,Title,Sellerid,PriceUSD,PriceBTC,Rating,Reviews,Origin,Destination,Category,Subcategory,Market,Date
0,"[28g / 1oz - Mega Sedative ""Wild Yam"" - Diosco...",b1g1mpact,None,0.063892,4.35,3,Czech Republic,Worldwide,Dissociatives,Dioscorea,SilkRoad2,2014-11-05


In [96]:
# Create empty list to populate
data = []

# Specify file directory with folders for different scraping sessions
datedir = "D:/Darkweb Data/dnmarchives/silkroad2.tar/silkroad2"

# Adds html ext to all files. Applies file cleaning function to all files from each scraping session and appends to one of two dataframes.
for scrapedate in os.listdir(datedir):
    directory = datedir +'/' + scrapedate + '/categories/'
    for foldername in os.listdir(directory):
        folder = directory + foldername # Concatenate directory path and filename for reading into cleanhtml function
#       converthtmlsr2(folder)
        for filename in os.listdir(folder):
            file = folder + '/' + filename
            if '2014-08=07' < scrapedate < '2014-08-15':
                filedf = cleanhtmlsr2(file)  # Extract dataframe of required variables from each file
                data.append(filedf)  # Append empty list with each dataframe generated. Results in list of dataframes.
                df = pd.concat(data)  # Concatenate list of dataframes into single dataframe 
##Works until mid-August?

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'D:/Darkweb Data/dnmarchives/silkroad2.tar/silkroad2/2014-11-06/categories/'

In [19]:
df

,Title,Sellerid,PriceUSD,PriceBTC,Rating,Reviews,Origin,Destination,Category,Subcategory,Market,Date
0,[Ray Ban Tech RB3460 001 Aviator/Flip Out/ Sun...,FoxyGirl,None,0.102405,None,None,China,Worldwide,Apparel,None,SilkRoad2,2014-06-23
1,Rolex - Submariner 2Tone YG/SS Black [Replica],RepAAA,None,0.250538,None,None,"Hong Kong, (China)",Worldwide,Apparel,None,SilkRoad2,2014-06-23
2,Rolex - Watch Box (AAA Grade Replica),RepAAA,None,0.168152,None,None,"Hong Kong, (China)",Worldwide,Apparel,None,SilkRoad2,2014-06-23
3,Ray Ban RB4140 Sunglasses Replica,FoxyGirl,None,0.056499,None,None,China,Worldwide,Apparel,None,SilkRoad2,2014-06-23
4,Canada Goose Chateau Parka Jackets Replica,FoxyGirl,None,0.229527,None,None,China,Worldwide,Apparel,None,SilkRoad2,2014-06-23
...,...,...,...,...,...,...,...,...,...,...,...,...
18,custom for mitch,ShroudWriter,None,0.000000,None,None,United States,Worldwide,Writing,None,SilkRoad2,2014-07-08
19,Academic Ghostwriting,profesorhouse,None,0.168889,None,None,Belgium,Undeclared,Writing,None,SilkRoad2,2014-07-08
20,[Unsaturated] Arbitrage [$100-$1000 ProfitWeek...,profesorhouse,None,0.168889,None,None,Belgium,Undeclared,Writing,None,SilkRoad2,2014-07-08
21,Make $500+ per day with CPC - Noob friendly,smart666tiger,None,0.337778,None,None,Undeclared,Worldwide,Writing,None,SilkRoad2,2014-07-08
